In [1]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

In [4]:
file_path = "/content/drive/My Drive/next-word-prediction/data/data1.txt"
doc = load_doc(file_path)
lines = doc.split('\n')

In [5]:
lines

['a guide',
 'guide to',
 'to word',
 'word embedding',
 'embedding with',
 'with gensim',
 'gensim model',
 'model handson',
 'handson graph',
 'graph neural',
 'neural networks',
 'networks with',
 'with pytorch',
 'pytorch pytorch',
 'pytorch geometric',
 'geometric how',
 'how to',
 'to use',
 'use in',
 'in python',
 'python databricks',
 'databricks how',
 'how to',
 'to save',
 'save files',
 'files in',
 'in csv',
 'csv on',
 'on your',
 'your local',
 'local computer',
 'computer a',
 'a stepbystep',
 'stepbystep implementation',
 'implementation of',
 'of gradient',
 'gradient descent',
 'descent and',
 'and backpropagation',
 'backpropagation an',
 'an easy',
 'easy introduction',
 'introduction to',
 'to sql',
 'sql for',
 'for data',
 'data scientists',
 'scientists hypothesis',
 'hypothesis testing',
 'testing visualized',
 'visualized introduction',
 'introduction to',
 'to latent',
 'latent matrix',
 'matrix factorization',
 'factorization recommender',
 'recommender sy

In [6]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [7]:
vocab_size

7761

In [8]:
sequences

[[3, 58],
 [58, 2],
 [2, 400],
 [400, 1409],
 [1409, 11],
 [11, 2310],
 [2310, 86],
 [86, 3472],
 [3472, 674],
 [674, 69],
 [69, 90],
 [90, 11],
 [11, 301],
 [301, 301],
 [301, 1731],
 [1731, 5],
 [5, 2],
 [2, 56],
 [56, 7],
 [7, 35],
 [35, 3473],
 [3473, 5],
 [5, 2],
 [2, 232],
 [232, 1410],
 [1410, 7],
 [7, 2311],
 [2311, 17],
 [17, 8],
 [8, 855],
 [855, 233],
 [233, 3],
 [3, 856],
 [856, 434],
 [434, 4],
 [4, 1732],
 [1732, 1173],
 [1173, 6],
 [6, 1411],
 [1411, 20],
 [20, 174],
 [174, 105],
 [105, 2],
 [2, 401],
 [401, 9],
 [9, 13],
 [13, 302],
 [302, 1412],
 [1412, 266],
 [266, 3474],
 [3474, 105],
 [105, 2],
 [2, 3475],
 [3475, 675],
 [675, 2312],
 [2312, 676],
 [676, 267],
 [267, 343],
 [343, 1733],
 [1733, 12],
 [12, 1],
 [1, 64],
 [64, 46],
 [46, 359],
 [359, 15],
 [15, 79],
 [79, 34],
 [34, 86],
 [86, 151],
 [151, 857],
 [857, 174],
 [174, 15],
 [15, 16],
 [16, 89],
 [89, 19],
 [19, 3476],
 [3476, 746],
 [746, 3477],
 [3477, 3478],
 [3478, 3479],
 [3479, 107],
 [107, 3],
 [3,

In [9]:
length = 2

In [10]:
print(len(sequences))
print(sequences[0][0])

48548
3


In [11]:
sequences = array(sequences)
sequences

array([[  3,  58],
       [ 58,   2],
       [  2, 400],
       ...,
       [  3,  74],
       [ 74,  47],
       [ 47, 673]])

In [12]:
sequences.size

97096

In [13]:
c=1
h=0
while(c!=0):
    limit = len(sequences)
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)

Total deletion:  0


In [14]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [15]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_1_1.log')
model.fit(X, y, batch_size=500, epochs=1000, callbacks=[csv_logger])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 50)             388050    
                                                                 
 gru (GRU)                   (None, 1, 100)            45600     
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7761)              783861    
                                                                 
Total params: 1,288,211
Trainable params: 1,288,211
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
98/98 [=====================

In [16]:
model.save('/content/drive/My Drive/next-word-prediction/models/model_1.h5')
dump(tokenizer, open(r'/content/drive/My Drive/next-word-prediction/models/tokenizer_1.pkl', 'wb'))